# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [134]:
import pandas as pd
import numpy as np

In [135]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [136]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [137]:
#verificando a estrutura de linhas e colunas do dataFrame
pg.shape

(20937, 26)

In [138]:
pg.drop_duplicates()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
20933,5150476,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
20934,5150480,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
20935,5150482,0,0,0,0,C,C,C,C,C,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [99]:
pg.shape

(20937, 26)

In [100]:
# 1) verificando dados faltantes
pg.isna().sum()

ID           0
mes_0        0
mes_1     1721
mes_10    3482
mes_11    3965
mes_12    3994
mes_13    4397
mes_14    4412
mes_15    4782
mes_16    4844
mes_17    5026
mes_18    5307
mes_19    5601
mes_2     1589
mes_20    5753
mes_21    5861
mes_22    6140
mes_23    6339
mes_24    6504
mes_3     2012
mes_4     2135
mes_5     2425
mes_6     2723
mes_7     3175
mes_8     3196
mes_9     3427
dtype: int64

In [101]:
#checabdo a base de dados 'propostas'
propostas.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [120]:
propostas.shape

(438557, 18)

In [139]:
#criando colunas com os valores default
pg.loc[pg['mes_0'].isin(['0','1','C']), 'default_mes0']  = 0
pg.loc[pg['mes_0'].isin(['2','3','4','5']), 'default_mes0']  = 1
pg.loc[pg['mes_1'].isin(['0','1','C']), 'default_mes1']  = 0
pg.loc[pg['mes_1'].isin(['2','3','4','5']), 'default_mes1']  = 1
pg.loc[pg['mes_2'].isin(['0','1','C']), 'default_mes2']  = 0
pg.loc[pg['mes_2'].isin(['2','3','4','5']), 'default_mes2']  = 1
pg.loc[pg['mes_3'].isin(['0','1','C']), 'default_mes3']  = 0
pg.loc[pg['mes_3'].isin(['2','3','4','5']), 'default_mes3']  = 1
pg.loc[pg['mes_4'].isin(['0','1','C']), 'default_mes4']  = 0
pg.loc[pg['mes_4'].isin(['2','3','4','5']), 'default_mes4']  = 1
pg.loc[pg['mes_5'].isin(['0','1','C']), 'default_mes5']  = 0
pg.loc[pg['mes_5'].isin(['2','3','4','5']), 'default_mes5']  = 1
pg.loc[pg['mes_6'].isin(['0','1','C']), 'default_mes6']  = 0
pg.loc[pg['mes_6'].isin(['2','3','4','5']), 'default_mes6']  = 1
pg.loc[pg['mes_7'].isin(['0','1','C']), 'default_mes7']  = 0
pg.loc[pg['mes_7'].isin(['2','3','4','5']), 'default_mes7']  = 1
pg.loc[pg['mes_8'].isin(['0','1','C']), 'default_mes8']  = 0
pg.loc[pg['mes_8'].isin(['2','3','4','5']), 'default_mes8']  = 1
pg.loc[pg['mes_9'].isin(['0','1','C']), 'default_mes9']  = 0
pg.loc[pg['mes_9'].isin(['2','3','4','5']), 'default_mes9']  = 1
pg.loc[pg['mes_10'].isin(['0','1','C']), 'default_mes10']  = 0
pg.loc[pg['mes_10'].isin(['2','3','4','5']), 'default_mes10']  = 1
pg.loc[pg['mes_11'].isin(['0','1','C']), 'default_mes11']  = 0
pg.loc[pg['mes_11'].isin(['2','3','4','5']), 'default_mes11']  = 1

In [140]:
pg.loc[pg['mes_12'].isin(['0','1','C']), 'default_mes12']  = 0
pg.loc[pg['mes_12'].isin(['2','3','4','5']), 'default_mes12']  = 1
pg.loc[pg['mes_13'].isin(['0','1','C']), 'default_mes13']  = 0
pg.loc[pg['mes_13'].isin(['2','3','4','5']), 'default_mes13']  = 1
pg.loc[pg['mes_14'].isin(['0','1','C']), 'default_mes14']  = 0
pg.loc[pg['mes_14'].isin(['2','3','4','5']), 'default_mes14']  = 1
pg.loc[pg['mes_15'].isin(['0','1','C']), 'default_mes15']  = 0
pg.loc[pg['mes_15'].isin(['2','3','4','5']), 'default_mes15']  = 1
pg.loc[pg['mes_16'].isin(['0','1','C']), 'default_mes16']  = 0
pg.loc[pg['mes_16'].isin(['2','3','4','5']), 'default_mes16']  = 1
pg.loc[pg['mes_17'].isin(['0','1','C']), 'default_mes17']  = 0
pg.loc[pg['mes_17'].isin(['2','3','4','5']), 'default_mes17']  = 1
pg.loc[pg['mes_18'].isin(['0','1','C']), 'default_mes18']  = 0
pg.loc[pg['mes_18'].isin(['2','3','4','5']), 'default_mes18']  = 1
pg.loc[pg['mes_19'].isin(['0','1','C']), 'default_mes19']  = 0
pg.loc[pg['mes_19'].isin(['2','3','4','5']), 'default_mes19']  = 1
pg.loc[pg['mes_20'].isin(['0','1','C']), 'default_mes20']  = 0
pg.loc[pg['mes_20'].isin(['2','3','4','5']), 'default_mes20']  = 1
pg.loc[pg['mes_21'].isin(['0','1','C']), 'default_mes21']  = 0
pg.loc[pg['mes_21'].isin(['2','3','4','5']), 'default_mes21']  = 1
pg.loc[pg['mes_22'].isin(['0','1','C']), 'default_mes22']  = 0
pg.loc[pg['mes_22'].isin(['2','3','4','5']), 'default_mes22']  = 1
pg.loc[pg['mes_23'].isin(['0','1','C']), 'default_mes23']  = 0
pg.loc[pg['mes_23'].isin(['2','3','4','5']), 'default_mes23']  = 1
pg.loc[pg['mes_24'].isin(['0','1','C']), 'default_mes24']  = 0
pg.loc[pg['mes_24'].isin(['2','3','4','5']), 'default_mes24']  = 1

In [141]:
pg.columns


Index(['ID', 'mes_0', 'mes_1', 'mes_10', 'mes_11', 'mes_12', 'mes_13',
       'mes_14', 'mes_15', 'mes_16', 'mes_17', 'mes_18', 'mes_19', 'mes_2',
       'mes_20', 'mes_21', 'mes_22', 'mes_23', 'mes_24', 'mes_3', 'mes_4',
       'mes_5', 'mes_6', 'mes_7', 'mes_8', 'mes_9', 'default_mes0',
       'default_mes1', 'default_mes2', 'default_mes3', 'default_mes4',
       'default_mes5', 'default_mes6', 'default_mes7', 'default_mes8',
       'default_mes9', 'default_mes10', 'default_mes11', 'default_mes12',
       'default_mes13', 'default_mes14', 'default_mes15', 'default_mes16',
       'default_mes17', 'default_mes18', 'default_mes19', 'default_mes20',
       'default_mes21', 'default_mes22', 'default_mes23', 'default_mes24'],
      dtype='object')

In [142]:
pg.dtypes

ID                 int64
mes_0             object
mes_1             object
mes_10            object
mes_11            object
mes_12            object
mes_13            object
mes_14            object
mes_15            object
mes_16            object
mes_17            object
mes_18            object
mes_19            object
mes_2             object
mes_20            object
mes_21            object
mes_22            object
mes_23            object
mes_24            object
mes_3             object
mes_4             object
mes_5             object
mes_6             object
mes_7             object
mes_8             object
mes_9             object
default_mes0     float64
default_mes1     float64
default_mes2     float64
default_mes3     float64
default_mes4     float64
default_mes5     float64
default_mes6     float64
default_mes7     float64
default_mes8     float64
default_mes9     float64
default_mes10    float64
default_mes11    float64
default_mes12    float64
default_mes13    float64


In [143]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,default_mes15,default_mes16,default_mes17,default_mes18,default_mes19,default_mes20,default_mes21,default_mes22,default_mes23,default_mes24
0,5001718,0,0,0,0,0,0,0,0,NaN,...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5001720,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
# 2) atribuindo uma coluna dos 'bons' e 'maus' pagadores ao longo de todos os 24 meses de desempenho
pg['classificação_pagador'] = np.where(pg.iloc[:,26:50].sum(axis=1) == 0, 'bom pagador','mau pagador')
                                                                   
                                                        

In [145]:
pg['classificação_pagador'].value_counts()

classificação_pagador
bom pagador    20515
mau pagador      422
Name: count, dtype: int64

In [152]:
#3) Marcando proponentes expostos ao risco de crédito. Foi feita a junção do Dataframe 'pg' ao dataframe das propostas assim 
# cada um está sendo marcado como 'bom' ou 'mau' pagador
# 4)Consolidando as informações (juntando os dois dataframes)
pg_selecao = pg.merge(propostas, on = 'ID', how = 'inner')


In [153]:
#checando a estrutura do novo dataframe
pg_selecao.shape

(16650, 69)

In [154]:
pg_selecao.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008806,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
1,5008810,0,NaN,C,C,C,C,C,C,C,...,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
2,5008811,0,NaN,C,C,C,C,C,C,C,...,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
3,5008825,0,0,0,0,0,0,0,0,0,...,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0
4,5008826,0,0,1,1,C,C,C,C,C,...,Married,House / apartment,-10669,-1103,1,0,0,0,Accountants,2.0


In [155]:
pg_selecao.columns

Index(['ID', 'mes_0', 'mes_1', 'mes_10', 'mes_11', 'mes_12', 'mes_13',
       'mes_14', 'mes_15', 'mes_16', 'mes_17', 'mes_18', 'mes_19', 'mes_2',
       'mes_20', 'mes_21', 'mes_22', 'mes_23', 'mes_24', 'mes_3', 'mes_4',
       'mes_5', 'mes_6', 'mes_7', 'mes_8', 'mes_9', 'default_mes0',
       'default_mes1', 'default_mes2', 'default_mes3', 'default_mes4',
       'default_mes5', 'default_mes6', 'default_mes7', 'default_mes8',
       'default_mes9', 'default_mes10', 'default_mes11', 'default_mes12',
       'default_mes13', 'default_mes14', 'default_mes15', 'default_mes16',
       'default_mes17', 'default_mes18', 'default_mes19', 'default_mes20',
       'default_mes21', 'default_mes22', 'default_mes23', 'default_mes24',
       'classificação_pagador', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL',

In [156]:
# 5) contagem dos valores default
pg_selecao['classificação_pagador'].value_counts()

classificação_pagador
bom pagador    16268
mau pagador      382
Name: count, dtype: int64